In [220]:
import os
import re
import zipfile
import glob
import pandas as pd
import sys
import time
import datetime
from yahoo_finance_api2 import share
from yahoo_finance_api2.exceptions import YahooFinanceError
from pandas_datareader import data


In [287]:
def get_share_price2(shareCode5):

    start = datetime.date(2022,2,1)
    end = datetime.date(2022,2,21)
    time.sleep(1) 
    df = data.DataReader("{0}.JP".format(shareCode5[0:4]),'stooq', start, end)
    df.head()
    
    return df

In [288]:
def get_share_price(shareCode5):
    my_share = share.Share("{0}.T".format(shareCode5[0:4]))
    symbol_data = None
    try:
        time.sleep(1)
        symbol_data = my_share.get_historical(
            share.PERIOD_TYPE_DAY, 1,
            share.FREQUENCY_TYPE_DAY, 1)
    except YahooFinanceError as e:
        print(e.message)

    df = pd.DataFrame(symbol_data)
    df.head()
    
    if df is not None:
        df['datetime'] = pd.to_datetime(df.timestamp, unit="ms")
        #日本時間へ変換
        df["datetime_JST"] = df["datetime"] + datetime.timedelta(hours=9)
        df.head()

    return df

In [289]:
def unzip_file(zip_dir,xbrl_file_expressions):
    zip_files = glob.glob(os.path.join(zip_dir, '*.zip'))

    number_of_zip_lists = len(zip_files)
    print("number_of_zip_lists：", number_of_zip_lists)

    xbrl_files = glob.glob(xbrl_file_expressions)
    return xbrl_files


In [290]:
def make_edinet_info_list(edinetcodedlinfo_filepath):
    edinet_info = pd.read_csv(edinetcodedlinfo_filepath, skiprows=1,
                                 encoding='cp932')
    edinet_info = edinet_info[["ＥＤＩＮＥＴコード", "提出者業種"]]
    edinet_info_list = edinet_info.values.tolist()
    return edinet_info_list

In [318]:
import sys
sys.path.append(r'./Arelle')
from Arelle.arelle import Cntlr

def make_edinet_company_info_list(xbrl_files,edinet_info_list):
    edinet_company_info_list = []

    for index, xbrl_file in enumerate(xbrl_files):
        edinet_code = ""  # EDINETCODE
        security_code = ""  # 証券コード
        filer_name_jp = ""  # 企業名
        industry_code = ""  # 業種
        operating_income = ""  # 営業利益
        current_assets = ""  # 流動資産
        current_liabilities = ""  #流動負債
        investments_and_other_assets = ""  #投資その他の資産
        noncurrent_liabilities = ""  #固定負債
        total_of_issued_share = ""  # 発行済株式数
        
        company_info_list = []  # 企業情報

        ctrl = Cntlr.Cntlr(logFileName='logToPrint')
        model_xbrl = ctrl.modelManager.load(xbrl_file)

        print(xbrl_file, ":", index + 1, "/", len(xbrl_files))

        for fact in model_xbrl.facts:
        
            #print(fact.concept.label)
            
            if fact.concept.qname.localName == 'EDINETCodeDEI':
                print("EDINETコード", fact.value)
                edinet_code = fact.value

                for code_name in edinet_info_list:
                    if code_name[0] == edinet_code:
                        print("業種",code_name[1])
                        industry_code = code_name[1]
                        break
            elif fact.concept.qname.localName == 'SecurityCodeDEI':
                print("証券コード", fact.value)
                security_code = fact.value

            elif fact.concept.qname.localName == 'FilerNameInJapaneseDEI':
                print("企業名", fact.value)
                filer_name_jp = fact.value

            elif fact.concept.qname.localName in ['TotalNumberOfIssuedSharesSummaryOfBusinessResults', 'NumberOfIssuedSharesAsOfFiscalYearEndIssuedSharesTotalNumberOfSharesEtc']:
                print("発行済株式数", fact.value)
                total_of_issued_share = fact.value

            elif fact.concept.qname.localName in ['OperatingIncome', 'OrdinaryIncomeLossSummaryOfBusinessResults']:
                ctx = fact.context
                if ctx is not None:
                    if ctx.id in [ 'CurrentYearDuration', 'CurrentYTDDuration', 'CurrentQuarterDuration', 'InterimDuration' ]:
                        print("営業利益(連結)", fact.value)
                        operating_income = fact.value
                    elif ctx.id in [ 'CurrentYTDDuration_NonConsolidatedMember', 'CurrentQuarterDuration_NonConsolidatedMember', 'CurrentYearDuration_NonConsolidatedMember', 'InterimDuration_NonConsolidatedMember' ]:
                        if operating_income == "":
                            print("営業利益(単体)", fact.value)
                            operating_income = fact.value

            elif fact.concept.label(preferredLabel=None, lang='ja', linkroleHint=None) == '流動資産':
                ctx = fact.context
                if ctx is not None:
                    if ctx.id in [ 'CurrentYearInstant', 'CurrentQuarterInstant', 'InterimInstant' ]:
                        print("流動資産(連結)", fact.value)
                        current_assets = fact.value
                    elif ctx.id in [ 'CurrentYearInstant_NonConsolidatedMember', 'CurrentQuarterInstant_NonConsolidatedMember', 'InterimInstant_NonConsolidatedMember' ]:
                        if current_assets == "":
                            print("流動資産(単体)", fact.value)
                            current_assets = fact.value

            elif fact.concept.label(preferredLabel=None, lang='ja', linkroleHint=None) == '流動負債':
                ctx = fact.context
                if ctx is not None:
                    if ctx.id in [ 'CurrentYearInstant', 'CurrentQuarterInstant', 'InterimInstant' ]:
                        print("流動負債(連結)", fact.value)
                        current_liabilities = fact.value
                    elif ctx.id in [ 'CurrentYearInstant_NonConsolidatedMember', 'CurrentQuarterInstant_NonConsolidatedMember', 'InterimInstant_NonConsolidatedMember' ]:
                        if current_liabilities == "":
                            print("流動負債(単体)", fact.value)
                            current_liabilities = fact.value

            elif fact.concept.qname.localName == 'InvestmentsAndOtherAssets':
                ctx = fact.context
                if ctx is not None:
                    if ctx.id in [ 'CurrentYearInstant', 'CurrentQuarterInstant', 'InterimInstant' ]:
                        print("投資その他の資産(連結)", fact.value)
                        investments_and_other_assets = fact.value
                    elif ctx.id in [ 'CurrentYearInstant_NonConsolidatedMember', 'CurrentQuarterInstant_NonConsolidatedMember', 'InterimInstant_NonConsolidatedMember' ]:
                        if investments_and_other_assets == "":
                            print("投資その他の資産(単体)", fact.value)
                            investments_and_other_assets = fact.value

            elif fact.concept.qname.localName == 'NoncurrentLiabilities':
                ctx = fact.context
                if ctx is not None:
                    if ctx.id in [ 'CurrentYearInstant', 'CurrentQuarterInstant', 'InterimInstant' ]:
                        print("固定負債(連結)", fact.value)
                        noncurrent_liabilities = fact.value
                    elif ctx.id in [ 'CurrentYearInstant_NonConsolidatedMember', 'CurrentQuarterInstant_NonConsolidatedMember', 'InterimInstant_NonConsolidatedMember' ]:
                        if noncurrent_liabilities == "":
                            print("固定負債(単体)", fact.value)
                            noncurrent_liabilities = fact.value


        print("")
        
        if security_code != "" and operating_income != "":
            price = get_share_price2(security_code)
            print(price)
            if price.size > 0:
                close_price = float(price['Close'][0])
            else:
                close_price = 0.0
        else:
            close_price = 0.0

        if operating_income == "":
            operating_income = 0.0
        else:
            operating_income = float(operating_income)

        if current_assets == "":
            current_assets = 0.0
        else:
            current_assets = float(current_assets)

        if current_liabilities == "":
            current_liabilities = 0.0
        else:
            current_liabilities = float(current_liabilities)

        if investments_and_other_assets == "":
            investments_and_other_assets = 0.0
        else:
            investments_and_other_assets = float(investments_and_other_assets)
        
        if noncurrent_liabilities == "":
            noncurrent_liabilities = 0.0
        else:
            noncurrent_liabilities = float(noncurrent_liabilities)

        if total_of_issued_share == "":
            total_of_issued_share = 1
        else:
            total_of_issued_share = float(total_of_issued_share)

        # 事業価値
        business_value = operating_income * 10.0
        
        # 財産価値
        property_value = current_assets - (current_liabilities * 1.2) + investments_and_other_assets
        
        # 株主価値
        shareholder_value = business_value + property_value - noncurrent_liabilities
        
        # 1株の価値
        share_unit_value = shareholder_value / total_of_issued_share
        
        # 安全率
        if close_price == 0:
            safety_ratio = 0
        else:
            safety_ratio = share_unit_value / close_price
        
        company_info_list.append(edinet_code)
        company_info_list.append(security_code)
        company_info_list.append(filer_name_jp)
        company_info_list.append(industry_code)
        company_info_list.append(operating_income)
        company_info_list.append(current_assets)
        company_info_list.append(current_liabilities)
        company_info_list.append(investments_and_other_assets)
        company_info_list.append(noncurrent_liabilities)
        company_info_list.append(business_value)
        company_info_list.append(property_value)
        company_info_list.append(shareholder_value)
        company_info_list.append(total_of_issued_share)
        company_info_list.append(share_unit_value)
        company_info_list.append(close_price)
        company_info_list.append(safety_ratio)
        
        print(company_info_list)
        print("")

        edinet_company_info_list.append(company_info_list)

    return pd.DataFrame(data=edinet_company_info_list, columns=['EDINETコード','証券コード','企業名','業種','営業利益','流動資産','流動負債','投資その他の資産','固定負債','事業価値','財産価値','株主価値','発行株数','1株価値','株価','安全率'])


In [319]:
def main():
    edinetcodedlinfo_filepath = r'./EdinetcodeDlInfo.csv'
    edinet_info_list = make_edinet_info_list(edinetcodedlinfo_filepath)

    zip_dir = r'./data/'
    xbrl_file_expressions = r'./data/XBRL/PublicDoc/*.xbrl'
    xbrl_files = unzip_file(zip_dir,xbrl_file_expressions)

    edinet_company_info_list = make_edinet_company_info_list(xbrl_files, edinet_info_list)
    print(edinet_company_info_list)

    edinet_company_info_list.to_csv('./share_value_result.csv',encoding="shift-jis")


In [ ]:
if __name__ == "__main__":
    main()

number_of_zip_lists： 6041
./data/XBRL/PublicDoc/jpcrp040300-q3r-001_E03641-000_2021-12-31_01_2022-02-10.xbrl : 1 / 6041
営業利益(連結) 15029000000
発行済株式数 138927858
発行済株式数 138927858
EDINETコード E03641
業種 銀行業
証券コード 85440
企業名 株式会社　京葉銀行

            Open  High  Low  Close  Volume
Date                                      
2022-02-18   533   535  524    524  345500
2022-02-17   542   547  532    545  255700
2022-02-16   533   539  525    537  259600
2022-02-15   543   546  520    523  402800
2022-02-14   533   545  533    540  230700
2022-02-10   546   546  535    543  390700
2022-02-09   526   542  524    539  875000
2022-02-08   522   527  515    527  387600
2022-02-07   511   529  511    522  709900
2022-02-04   491   503  491    501  240500
2022-02-03   488   493  486    491  129900
2022-02-02   483   489  479    489  204600
2022-02-01   474   484  474    476  209100
['E03641', '85440', '株式会社\u3000京葉銀行', '銀行業', 15029000000.0, 0.0, 0.0, 0.0, 0.0, 150290000000.0, 0.0, 150290000000.0, 138927858.0,

./data/XBRL/PublicDoc/jpcrp040300-q3r-001_E04011-000_2021-09-30_01_2021-11-11.xbrl : 9 / 6041
営業利益(単体) 2349298000
発行済株式数 12940900
発行済株式数 12940900
発行済株式数 12940900
発行済株式数 12940900
発行済株式数 12940900
流動資産(単体) 16189832000
投資その他の資産(単体) 4198325000
流動負債(単体) 6933242000
固定負債(単体) 14903952000
EDINETコード E04011
業種 不動産業
証券コード 89140
企業名 エリアリンク株式会社

            Open  High   Low  Close  Volume
Date                                       
2022-02-18  1315  1353  1300   1322   42700
2022-02-17  1353  1370  1325   1331   46100
2022-02-16  1415  1422  1294   1353  103300
2022-02-15  1485  1485  1400   1407   48100
2022-02-14  1479  1500  1471   1479   20000
2022-02-10  1504  1547  1500   1519   49600
2022-02-09  1490  1504  1478   1501   13200
2022-02-08  1496  1498  1482   1498   10900
2022-02-07  1510  1510  1471   1496   13000
2022-02-04  1505  1512  1478   1512   27700
2022-02-03  1479  1515  1458   1509   44400
2022-02-02  1436  1461  1432   1460   15300
2022-02-01  1443  1470  1432   1439   30600
['E0401

営業利益(連結) 921111000
発行済株式数 15051227
発行済株式数 15051227
流動資産(連結) 17021502000
投資その他の資産(連結) 2995830000
流動負債(連結) 9310710000
固定負債(連結) 3335360000
営業利益(連結) 795374000
EDINETコード E27868
業種 鉄鋼
証券コード 56980
企業名 株式会社エンビプロ・ホールディングス

            Open  High   Low  Close  Volume
Date                                       
2022-02-18  1286  1324  1280   1311   95700
2022-02-17  1318  1324  1286   1306  110400
2022-02-16  1331  1347  1295   1298  146500
2022-02-15  1328  1347  1306   1311  104300
2022-02-14  1329  1340  1292   1331  207200
2022-02-10  1390  1399  1331   1359  294200
2022-02-09  1400  1411  1349   1382  340400
2022-02-08  1486  1510  1395   1400  629300
2022-02-07  1655  1694  1622   1670  286600
2022-02-04  1600  1649  1576   1642  141500
2022-02-03  1654  1654  1581   1597  134700
2022-02-02  1596  1626  1578   1619   96300
2022-02-01  1598  1627  1520   1537  170700
['E27868', '56980', '株式会社エンビプロ・ホールディングス', '鉄鋼', 795374000.0, 17021502000.0, 9310710000.0, 2995830000.0, 3335360000.0, 79537400

./data/XBRL/PublicDoc/jpcrp040300-q3r-001_E00679-000_2021-12-31_01_2022-02-14.xbrl : 24 / 6041
営業利益(連結) 1241000000
発行済株式数 28794294
発行済株式数 28794294
流動資産(連結) 29794000000
投資その他の資産(連結) 5076000000
流動負債(連結) 19627000000
固定負債(連結) 10741000000
営業利益(連結) 796000000
営業利益(連結) 796000000
EDINETコード E00679
業種 その他製品
証券コード 79870
企業名 ナカバヤシ株式会社

            Open  High  Low  Close  Volume
Date                                      
2022-02-18   502   508  502    508   25600
2022-02-17   512   512  505    508   47200
2022-02-16   516   519  512    512   54200
2022-02-15   515   519  515    516   38800
2022-02-14   516   517  511    517   34200
2022-02-10   516   519  514    519   27500
2022-02-09   515   517  514    516   25800
2022-02-08   516   518  515    517   18000
2022-02-07   513   518  513    516   21200
2022-02-04   512   515  510    513   21800
2022-02-03   517   517  512    513   18000
2022-02-02   512   518  511    518   23400
2022-02-01   512   515  509    509   23400
['E00679', '79870', 'ナカバヤシ株式会社

./data/XBRL/PublicDoc/jpcrp040300-q2r-001_E00475-000_2021-09-30_01_2021-11-12.xbrl : 31 / 6041
営業利益(連結) 340522000
発行済株式数 13056198
発行済株式数 13056198
流動資産(連結) 15142515000
投資その他の資産(連結) 2291195000
流動負債(連結) 2718187000
固定負債(連結) 344718000
営業利益(連結) 307535000
営業利益(連結) 307535000
EDINETコード E00475
業種 食料品
証券コード 28120
企業名 焼津水産化学工業株式会社

            Open  High  Low  Close  Volume
Date                                      
2022-02-18   915   923  908    916    5800
2022-02-17   920   920  912    912    3800
2022-02-16   923   923  920    920    3500
2022-02-15   921   922  915    919    6100
2022-02-14   920   920  912    918    5100
2022-02-10   910   916  910    915    6900
2022-02-09   909   910  907    910    5000
2022-02-08   907   911  906    909    6000
2022-02-07   905   913  905    907   20200
2022-02-04   920   923  913    917    7100
2022-02-03   925   930  920    920    2900
2022-02-02   928   932  919    932    4700
2022-02-01   923   931  910    918    6500
['E00475', '28120', '焼津水産化学工業株式会社

...

./data/XBRL/PublicDoc/jpcrp040300-q3r-001_E02254-000_2021-12-31_01_2022-02-10.xbrl : 356 / 6041
営業利益(連結) -1812850000
発行済株式数 26863974
発行済株式数 26863974
流動資産(連結) 71550078000
投資その他の資産(連結) 10343906000
流動負債(連結) 69547797000
固定負債(連結) 11758427000
営業利益(連結) -1520183000
EDINETコード E02254
業種 輸送用機器
証券コード 74080
企業名 株式会社ジャムコ

            Open  High  Low  Close  Volume
Date                                      
2022-02-18   819   826  805    819   43300
2022-02-17   846   846  820    822   48600
2022-02-16   837   857  835    839   48300
2022-02-15   826   841  812    820   53700
2022-02-14   831   840  808    823   78200
2022-02-10   888   902  841    846  122700
2022-02-09   858   899  853    877   79200
2022-02-08   843   865  843    858   35800
2022-02-07   833   839  824    831   47900
2022-02-04   832   845  805    842   49300
2022-02-03   825   836  820    832   41300
2022-02-02   798   850  798    849   57300
2022-02-01   820   823  795    796   68100
['E02254', '74080', '株式会社ジャムコ', '輸送用機器', -152

            Open  High   Low  Close   Volume
Date                                        
2022-02-18  1228  1279  1207   1268   481000
2022-02-17  1350  1353  1261   1261   989400
2022-02-16  1255  1315  1244   1303  1041700
2022-02-15  1246  1255  1198   1212   685100
2022-02-14  1282  1297  1223   1246   968000
2022-02-10  1437  1491  1300   1322  2976100
2022-02-09  1186  1212  1153   1197   597600
2022-02-08  1157  1229  1151   1167   613800
2022-02-07  1275  1275  1156   1205   661300
2022-02-04  1240  1300  1228   1278   695900
2022-02-03  1222  1314  1221   1270   784700
2022-02-02  1231  1326  1231   1305   761900
2022-02-01  1252  1310  1206   1219   932300
['E34973', '41650', '株式会社プレイド', '情報・通信業', 170616000.0, 4956539000.0, 1640271000.0, 388593000.0, 390219000.0, 1706160000.0, 3376806800.0, 4692747800.0, 37890900.0, 123.84893998295105, 1268.0, 0.0976726656016964]

./data/XBRL/PublicDoc/jpcrp040300-q3r-001_E35170-000_2021-10-31_01_2021-12-15.xbrl : 364 / 6041
営業利益(連結) 52321600

./data/XBRL/PublicDoc/jpcrp040300-q3r-001_E37141-000_2021-12-31_01_2022-02-10.xbrl : 370 / 6041
営業利益(単体) 1111988000
発行済株式数 4466300
発行済株式数 3813300
発行済株式数 4466300
発行済株式数 4466300
流動資産(単体) 9183693000
投資その他の資産(単体) 306363000
流動負債(単体) 4038425000
固定負債(単体) 41285085000
EDINETコード E37141
業種 不動産業
証券コード 29930
企業名 株式会社長栄

            Open  High   Low  Close  Volume
Date                                       
2022-02-18  1607  1665  1600   1645   16100
2022-02-17  1627  1629  1607   1608    4900
2022-02-16  1601  1610  1592   1610    6500
2022-02-15  1592  1628  1592   1601   11000
2022-02-14  1582  1600  1576   1593    6700
2022-02-10  1575  1585  1572   1575    3200
2022-02-09  1584  1588  1573   1574    5300
2022-02-08  1592  1592  1570   1577    3500
2022-02-07  1590  1603  1580   1580    3500
2022-02-04  1579  1599  1579   1581    2500
2022-02-03  1576  1612  1576   1578    7100
2022-02-02  1584  1594  1581   1588    6400
2022-02-01  1565  1594  1565   1577    3900
['E37141', '29930', '株式会社長栄', '

            Open  High  Low  Close  Volume
Date                                      
2022-02-18   265   275  265    272   25600
2022-02-17   275   278  265    271   34400
2022-02-16   272   275  266    275   16800
2022-02-15   279   279  266    268   60400
2022-02-14   271   276  270    272   30100
2022-02-10   280   284  273    277   75800
2022-02-09   291   291  284    291   35100
2022-02-08   289   289  278    285   34900
2022-02-07   286   288  284    286   10300
2022-02-04   291   291  284    286   36200
2022-02-03   292   292  288    292   17800
2022-02-02   289   294  288    294    5100
2022-02-01   294   294  287    287   54600
['E00868', '49600', 'ケミプロ化成株式会社', '化学', 305108000.0, 8664062000.0, 7102996000.0, 290316000.0, 2536771000.0, 3051080000.0, 430782800.0, 945091800.0, 16623613.0, 56.85237017969559, 272.0, 0.20901606683711615]

./data/XBRL/PublicDoc/jpcrp040300-q2r-001_E03129-000_2021-09-30_01_2021-11-12.xbrl : 378 / 6041
営業利益(連結) 3234000000
発行済株式数 47911023
発行済株式数 47911023

            Open  High   Low  Close   Volume
Date                                        
2022-02-18  1295  1321  1294   1314  1332700
2022-02-17  1305  1318  1303   1310  1363000
2022-02-16  1339  1339  1304   1309  1884400
2022-02-15  1324  1330  1305   1318  2481700
2022-02-14  1314  1345  1314   1332  2307600
2022-02-10  1323  1347  1317   1345  3856800
2022-02-09  1291  1339  1269   1308  7229700
2022-02-08  1245  1265  1243   1258  2310500
2022-02-07  1236  1249  1221   1247  1771600
2022-02-04  1245  1255  1232   1244  1524800
2022-02-03  1258  1259  1238   1253  2254200
2022-02-02  1262  1284  1262   1276  1700700
2022-02-01  1279  1279  1258   1265  1494900
['E01773', '67530', 'シャープ株式会社', '電気機器', 63887000000.0, 1235904000000.0, 854653000000.0, 255286000000.0, 679014000000.0, 638870000000.0, 465606400000.0, 425462400000.0, 611952858.0, 695.2535549723668, 1314.0, 0.5291122944995181]

./data/XBRL/PublicDoc/jpcrp040300-q3r-001_E02824-000_2021-12-31_01_2022-02-10.xbrl : 385 / 6041
